# Part 1

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'html.parser')
table = soup.table

In [3]:
data = pd.read_html(str(table))
df=pd.DataFrame(data[0])
df=df[df['Borough'] != "Not assigned"]
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [4]:
df['Neighborhood']=[', '.join(map(str, l)) for l in df['Neighborhood'].str.split(' / ')]
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


The first thing I did is to only work with Borough that has a value different form __Not assigned__  
Then I convert the Neighborhood values form this *Regent Park / Harbourfront* to this *Regent Park, Harbourfront*

In [5]:
df.shape

(103, 3)

# Part 2

In [6]:
df_lat_lon=pd.read_csv("https://cocl.us/Geospatial_data")

In [7]:
df_lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
newdf=df.join(df_lat_lon.set_index('Postal Code'),on="Postal code")

In [9]:
newdf.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


I use the csv with the latitde and longitude of all the neighborhoods. Then __join__ both dataframes into a new one with all the complete data

# Part 3

In [10]:
from geopy.geocoders import Nominatim
! pip install folium
import folium

     |████████████████████████████████| 92kB 15.8MB/s eta 0:00:01


### Get the latitude and longitud of Ontario

In [11]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ontario_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Ontario are 43.6534817, -79.3839347.


### Create the map with all the points

In [12]:
map_ontario = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for borough,neighborhood,lat,lng in zip(newdf["Borough"],newdf["Neighborhood"],newdf["Latitude"],newdf["Longitude"]):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ontario)  

In [13]:
map_ontario

### Foursquare Credentials and Version

In [14]:
# The code was removed by Watson Studio for sharing.

### Exploring the first neighborhood in the dataframe

In [93]:
n_latitude = newdf.loc[0, 'Latitude'] # neighborhood latitude value
n_longitude = newdf.loc[0, 'Longitude'] # neighborhood longitude value

n_name = newdf.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(n_name, 
                                                               n_latitude, 
                                                               n_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


### Get the 100 venues that are in Parkwoods within a radius of 500 meters.

In [95]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    n_latitude, 
    n_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=PHG4C3ZYV5OX1VJLVW52SJNCO5RU24LLN0RQ2WG2SUVOV4FZ&client_secret=3ZX1UBPUUN5T1AXPCIQFW1STZOK5LZZKRK53NMNERCLDXIH2&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

Sending the get request

In [96]:
results = requests.get(url).json()

In [90]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Create dataframe with the response

In [117]:
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114
2,TTC stop - 44 Valley Woods,Bus Stop,43.755402,-79.333741


### Explore all the Neighborhoods and create a dataframe

In [20]:
LIMIT = 100
radius = 500
venues_list=[]
for name, lat, lng in zip(newdf['Neighborhood'],newdf['Latitude'],newdf['Longitude']):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)
    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']
    venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venues])

nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = [ 'Neighborhood', 'Neighborhood Latitude', 
                          'Neighborhood Longitude', 'Venue', 
                          'Venue Latitude', 'Venue Longitude', 'Venue Category']

In [85]:
nearby_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


### Transform categorical values (Vanue Category) to numerical values using one hot encoding

In [78]:
ontario_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
ontario_onehot['Neighborhood Name'] = nearby_venues[['Neighborhood']]

# move neighborhood column to the first column
fixed_columns = [ontario_onehot.columns[-1]] + list(ontario_onehot.columns[:-1])
ontario_onehot = ontario_onehot[fixed_columns]

ontario_onehot.head()

,Neighborhood Name,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio


#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [77]:
ontario_grouped = ontario_onehot.groupby("Neighborhood Name").mean().reset_index()
ontario_grouped.head()

,Neighborhood Name,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio


#### Exploring the top 10 vanues from each neighborhood checking that their frequency should be higher than 0

In [24]:
num_top_venues = 10

for hood in ontario_grouped['Neighborhood Name']:
    print("----"+hood+"----")
    temp = ontario_grouped[ontario_grouped['Neighborhood Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp[temp['freq']>0].sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1         Chinese Restaurant  0.25
2  Latin American Restaurant  0.25
3                     Lounge  0.25


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place   0.2
1  Athletics & Sports   0.1
2         Coffee Shop   0.1
3        Dance Studio   0.1
4                 Gym   0.1
5            Pharmacy   0.1
6                 Pub   0.1
7      Sandwich Place   0.1
8        Skating Rink   0.1


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                       Bank  0.11
1                Coffee Shop  0.11
2                   Pharmacy  0.05
3           Sushi Restaurant  0.05
4                Supermarket  0.05
5              Shopping Mall  0.05
6             Sandwich Place  0.05
7                 Restaurant  0.05
8                Pizza Place  0.05
9  Middle Eastern Restaurant  0.05


----Bayview Village----
                 venue  f

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    freq = row_categories_sorted.where(row_categories>0).count()
    if freq > num_top_venues:
        return row_categories_sorted.index.values[0:num_top_venues]
    else:
        return list(row_categories_sorted.index.values[0:freq])+["" for i in range(num_top_venues-freq)]

In [61]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ontario_grouped['Neighborhood Name']

for ind in np.arange(ontario_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ontario_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Breakfast Spot,Lounge,Chinese Restaurant,,,,,,
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Dance Studio,Pub,Athletics & Sports,Skating Rink,Gym,
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Shopping Mall,Sandwich Place,Bridal Shop,Diner,Restaurant,Ice Cream Shop,Deli / Bodega
3,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,,,,,,
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Restaurant,Pizza Place,Sushi Restaurant,Thai Restaurant,Liquor Store,Indian Restaurant,Pub


## Cluster Neighborhoods using K-means

In [53]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

ontario_grouped_clustering = ontario_grouped.drop('Neighborhood Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ontario_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 4, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       3, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1], dtype=int32)

In [59]:
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ontario_merged = newdf

ontario_merged = ontario_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
ontario_merged.head(15)

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Bus Stop,Food & Drink Shop,,,,,,,
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Coffee Shop,Grocery Store,Hockey Arena,Portuguese Restaurant,,,,,,
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Pub,Park,Bakery,Theater,Restaurant,Café,Breakfast Spot,Hotel,Beer Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Carpet Store,Miscellaneous Shop,Event Space,Coffee Shop,Boutique,
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Sushi Restaurant,Diner,Juice Bar,Bank,Bar,Café,Spa,Beer Bar,Yoga Studio
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1.0,Fast Food Restaurant,,,,,,,,,
7,M3B,North York,Don Mills,43.745906,-79.352188,1.0,Japanese Restaurant,Beer Store,Coffee Shop,Gym,Restaurant,Asian Restaurant,Italian Restaurant,Caribbean Restaurant,Sandwich Place,Bike Shop
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1.0,Pizza Place,Pharmacy,Fast Food Restaurant,Café,Breakfast Spot,Athletics & Sports,Intersection,Pet Store,Bank,Gym / Fitness Center
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Tea Room,Diner


#### There are some neighborhoods with no venues nearby or that data is missing (NaN). Drop all those values

In [110]:
ontario_final=ontario_merged.dropna(axis=0)
ontario_final["Cluster Labels"]=ontario_final["Cluster Labels"].astype(int)
ontario_final.head(15)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Park,Bus Stop,Food & Drink Shop,,,,,,,
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Coffee Shop,Grocery Store,Hockey Arena,Portuguese Restaurant,,,,,,
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Pub,Park,Bakery,Theater,Restaurant,Café,Breakfast Spot,Hotel,Beer Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Carpet Store,Miscellaneous Shop,Event Space,Coffee Shop,Boutique,
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Diner,Juice Bar,Bank,Bar,Café,Spa,Beer Bar,Yoga Studio
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1,Fast Food Restaurant,,,,,,,,,
7,M3B,North York,Don Mills,43.745906,-79.352188,1,Japanese Restaurant,Beer Store,Coffee Shop,Gym,Restaurant,Asian Restaurant,Italian Restaurant,Caribbean Restaurant,Sandwich Place,Bike Shop
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1,Pizza Place,Pharmacy,Fast Food Restaurant,Café,Breakfast Spot,Athletics & Sports,Intersection,Pet Store,Bank,Gym / Fitness Center
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Tea Room,Diner
10,M6B,North York,Glencairn,43.709577,-79.445073,2,Pizza Place,Pub,Japanese Restaurant,,,,,,,


### Finally, let's visualize the resulting clusters

In [45]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [118]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ontario_final['Latitude'], ontario_final['Longitude'], ontario_final['Neighborhood'], ontario_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1

In [116]:
ontario_final.loc[ontario_final['Cluster Labels'] == 0]

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Park,Bus Stop,Food & Drink Shop,,,,,,,
16,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,0,Park,Hockey Arena,Field,Trail,,,,,,
21,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,0,Park,Women's Store,Pool,,,,,,,
35,M4J,East York,East Toronto,43.685347,-79.338106,0,Park,Coffee Shop,Metro Station,Convenience Store,,,,,,
49,M6L,North York,"North Park, Maple Leaf Park, Upwood Park",43.713756,-79.490074,0,Park,Construction & Landscaping,Bakery,,,,,,,
61,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Swim School,Park,Bus Line,,,,,,,
64,M9N,York,Weston,43.706876,-79.518188,0,Park,Convenience Store,,,,,,,,
66,M2P,North York,York Mills West,43.752758,-79.400049,0,Park,Bank,Convenience Store,,,,,,,
85,M1V,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",43.815252,-79.284577,0,Coffee Shop,Park,Playground,,,,,,,
91,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Trail,Playground,,,,,,,


#### Cluster 2

In [123]:
ontario_final.loc[ontario_final['Cluster Labels'] == 1]

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Coffee Shop,Grocery Store,Hockey Arena,Portuguese Restaurant,,,,,,
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Pub,Park,Bakery,Theater,Restaurant,Café,Breakfast Spot,Hotel,Beer Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Carpet Store,Miscellaneous Shop,Event Space,Coffee Shop,Boutique,
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Diner,Juice Bar,Bank,Bar,Café,Spa,Beer Bar,Yoga Studio
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1,Fast Food Restaurant,,,,,,,,,
7,M3B,North York,Don Mills,43.745906,-79.352188,1,Japanese Restaurant,Beer Store,Coffee Shop,Gym,Restaurant,Asian Restaurant,Italian Restaurant,Caribbean Restaurant,Sandwich Place,Bike Shop
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1,Pizza Place,Pharmacy,Fast Food Restaurant,Café,Breakfast Spot,Athletics & Sports,Intersection,Pet Store,Bank,Gym / Fitness Center
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Tea Room,Diner
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1,Bar,History Museum,,,,,,,,
13,M3C,North York,Don Mills,43.725900,-79.340923,1,Japanese Restaurant,Beer Store,Coffee Shop,Gym,Restaurant,Asian Restaurant,Italian Restaurant,Caribbean Restaurant,Sandwich Place,Bike Shop


#### Cluster 3

In [120]:
ontario_final.loc[ontario_final['Cluster Labels'] == 2]

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,M6B,North York,Glencairn,43.709577,-79.445073,2,Pizza Place,Pub,Japanese Restaurant,,,,,,,
50,M9L,North York,Humber Summit,43.756303,-79.565963,2,Pizza Place,,,,,,,,,


#### Cluster 4

In [121]:
ontario_final.loc[ontario_final['Cluster Labels'] == 3]

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,M5N,Central Toronto,Roselawn,43.711695,-79.416936,3,Garden,,,,,,,,,


#### Cluster 5

In [124]:
ontario_final.loc[ontario_final['Cluster Labels'] == 4]

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,4,Convenience Store,Playground,,,,,,,,
83,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,4,Playground,,,,,,,,,
